## Importing all the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import os
from collections import defaultdict

In [2]:
# Define the mode: DEBUG = 1 => Print all the info, DEBUG = 0 => Skip printing
DEBUG = False

# "./thesis/checkpoints/chkp_augmentation_mt.h5" -> MIXTURE TECHNIQUES
# "./thesis/checkpoints/chkp_40_final.h5" -> ORIGINAL NET
# "./thesis/checkpoints/chkp_augmentation_awn.h5" -> DATA AUGMENTATION
# "./thesis/checkpoints/chkp_augmentation_awn_p.h5" -> DATA AUGMENTATION - PARALLEL NET [SET MODE = 1]

# Define the models to be used (None if no model must be used)
modelName = "./thesis/checkpoints/chkp_40_final.h5"
model1Name = "./thesis/checkpoints/chkp_augmentation_awn_p.h5"

# Define whether to parallelize networks: MODE = 0 => only the first model, MODE = 1 => Use also the parallel network (model1)
MODE = 1

### Function to be used for computing the parameters of the curves

In [3]:
def compute_params(curve):
    val_0 = np.mean(curve[55-5:55+5])
    val_1 = np.mean(curve[100-10:100+10])
    val_2 = np.mean(curve[200-10:200+10])
    val_3 = np.mean(curve[350-10:350+10])
    val_4 = np.mean(curve[630-10:630+10])
    
    return val_0, val_1, val_2, val_3, val_4

## Dataset MNG

In [4]:
import pickle

# Import the complete dataset
with open('./thesis/XX_40.pkl', 'rb') as f:
    XX = pickle.load(f)

if DEBUG is True:
    print (XX.shape)

In [5]:
filters = np.zeros((len(XX), ), dtype=bool)
filters[:13650 + 8000] = np.ones((13650 + 8000, ))

if DEBUG is True:
    np.sum(filters)

In [6]:
X = XX[filters].astype(np.float32)

if DEBUG is True:
    print(X.shape)
    print(X[3000:5000,:])

In [7]:
# Shuffle data in test dataset
np.random.seed(0)
np.random.shuffle(X)

In [8]:
# Split the dataset, in order to obtain the test and validation dataset
train_ts = int(X.shape[0]*0.6)
valid_ts = int(X.shape[0]*0.8)

x_train = X[:train_ts, :-2]
y_train = X[:train_ts, -2:]

x_valid = X[train_ts:valid_ts, :-2]
y_valid = X[train_ts:valid_ts, -2:]

x_test = X[valid_ts:, :-2]
y_test = X[valid_ts:, -2:]

# minmax normalization
xmin = np.min(x_train, axis=0)
xmax = np.max(x_train, axis=0)

x_train = (x_train - xmin) / (xmax - xmin)
x_valid = (x_valid - xmin) / (xmax - xmin)
x_test  = (x_test  - xmin) / (xmax - xmin)

if DEBUG is True:
    print (x_test)
    
ymax = np.max(y_train, axis=0)
ymin = np.min(y_train, axis=0)

y_train = (y_train - ymin) / (ymax - ymin)
y_valid = (y_valid - ymin) / (ymax - ymin)
y_test  = (y_test  - ymin) / (ymax - ymin)

if DEBUG is True:
    print('normalization parameters')
    print('xmin: \t' + str(xmin))
    print('xmax: \t' + str(xmax))
    print('ymin: \t' + str(ymin))
    print('ymax: \t' + str(ymax))
    print('data shapes')
    print(x_train.shape)
    print(x_valid.shape)
    print(x_test.shape)
    

In [9]:
# Function to be used for denormalizing data given by the model
def denormalize(y_pred):
    return y_pred*(ymax-ymin) + ymin

## Model robustness functions

In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_absolute_error
from keras import backend as K
from keras.losses import mean_absolute_error as keras_mae

In [11]:
# Wighted MAE function
def weightedMAE(y_true, y_pred):
    mae = keras_mae(y_true, y_pred)
    return mae*( (1 - K.mean(y_true))**2 ) 

In [12]:
# Classical MAE function
def mae(y_true, y_pred):
    errors = np.mean(np.abs((y_true - y_pred)))
    return errors

In [13]:
# MAPE Function
def mean_absolute_percentage_error(y_true, y_pred): 
    errors = np.abs((y_true - y_pred) / (y_true + 0.00001)) * 100
    return np.mean(errors)

In [14]:
# APE Function
def absolute_percentage_error(y_true, y_pred): 
    errors = np.abs((y_true - y_pred) / (y_true + 0.00001)) * 100
    return errors

### Load the models

In [15]:
from keras.utils.vis_utils import plot_model

model = keras.models.load_model(modelName, custom_objects={'weightedMAE': weightedMAE})

if (DEBUG is True):
    plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

if model1Name is not None:
    model1 = keras.models.load_model(model1Name, custom_objects={'weightedMAE': weightedMAE})

## Dataset Preparation

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Open the pickle files in order to have the curves to be altered
with open('./thesis/X_robustness.pkl', 'rb') as f:
    X = pickle.load(f)
    
with open('./thesis/onde_robustness.pkl', 'rb') as f:
    onde_raw = pickle.load(f)

if DEBUG is True:
    print(X.shape)
    print(onde_raw.shape)

In [17]:
# Define the dataframe
columns = ['TBlood', 'v0', 'v1', 'v2', 'v3', 'v4', 'pO2_ABL1_37', 'pO2_ABL1']
df = pd.DataFrame(X, columns=columns)

if DEBUG is True:
    print(df.shape)

## Robustness functions

### 1. UU Robustness

In [18]:
from scipy.integrate import quad

def integrand_h(x, x0, err):
    return err[int(x)]/x0

def UU_robustness(mape, theta, x0):
    err = theta-mape
    h = np.heaviside(err,0)
    
    r = quad(integrand_h, 0, x0, args=(x0,h))
    return r[0]

### 2. UL Robustness

In [19]:
from scipy.integrate import quad

def integrand_b(x, x0, err):
    if (err[int(x)] > 0):
        return err[int(x)]/x0
    else:
        return 0

def UL_robustness(mape, theta, x0):
    err = theta-mape
    
    r = quad(integrand_b, 0, x0, args=(x0,err)) 
    return r[0] / theta

### 3. LU Robustness

In [20]:
from scipy.integrate import quad

def integrand_w(x, x0, err):
    return err[int(x)] * (x0 - x)

def LU_robustness(mape, theta, x0):
    err = theta-mape
    h = np.heaviside(err,0)
    
    r = quad(integrand_w, 0, x0, args=(x0,h)) 
    return r[0] / (0.5 * x0 * x0)

### 4. LL Robustness

In [21]:
from scipy.integrate import quad

def integrand_LL(x, x0, err):
    if (err[int(x)] > 0):
        return err[int(x)] * (x0 - x)
    else:
        return 0

def LL_robustness(mape, theta, x0):
    err = theta-mape
    
    r = quad(integrand_LL, 0, x0, args=(x0,err)) 
    return r[0] / (0.5 * theta * x0 * x0)

## Robustness ANALYSIS

In [22]:
# Threshold
theta = 10

### 1. Cut of the end of the curve 

In [23]:
import numpy as np

# Configuration params
l_A = 620
u_A = 640
theta = 10

# Function to be used for cutting the end of the curves
def curve_end_cut(onde_raw, start):
    onde_new = np.copy(onde_raw)
    onde_new[:,start:]=0.0
    return onde_new

In [24]:
# Alteration on all the curves and data analysis

# Prepare the resulting vectors as empty ones 
mape = np.empty(((u_A-l_A)+1, 2), dtype=np.float32)

# Apply all the levels of alteration
for j in range((u_A-l_A)+1):
    
    # Alter all the curves
    onde_new = curve_end_cut(onde_raw, u_A-j)    

    # Compute the parameters (5 average values) from each curve
    params = np.empty((len(onde_new), 5), dtype=np.float32)
    for i in range(len(onde_new)):
        try:
            params[i] = np.array(compute_params(onde_new[i]))
            assert np.any(np.isnan(params[i])) == False
        except:
            print("Parameter " + str(i) + " cannot be computed")
            continue
            
    # Test set + normalization
    new_data  = np.concatenate([df.TBlood.values.reshape(-1,1),params], axis = 1)
    test  = (new_data  - xmin) / (xmax - xmin) 

    # Prediction using the models
    if MODE == 0:
        pred = model.predict(test)
        y_pred_denorm  = denormalize(pred)
    else:
        # Using the parallel network as well
        y_pred_t0 = model.predict(test)
        y_pred_t1 = model1.predict(test)

        y_pred_t0_denorm  = denormalize(y_pred_t0)
        y_pred_t1_denorm  = denormalize(y_pred_t1)
        y_pred_denorm = (y_pred_t0_denorm + y_pred_t1_denorm)/2

    # Save the true values for evaluation
    pO2_true = np.concatenate([df.pO2_ABL1_37.values.reshape(-1,1), df.pO2_ABL1.values.reshape(-1,1)], axis=1)
   
    # Compute the MAPE
    mape[j,0]=mean_absolute_percentage_error(pO2_true[:, 0], y_pred_denorm[:, 0])
    mape[j,1]=mean_absolute_percentage_error(pO2_true[:, 1], y_pred_denorm[:, 1])

In [25]:
# Estimating the robustness w.r.t. Current pO2
ideal = np.ones((mape.shape[0])) * mape[0,1]
real = mape[:,1]
real_shape = real.shape[0]
ideal_shape = ideal.shape[0]

heav = UU_robustness(real, theta, real_shape)
print('UU Robustness [Cut of the end of the curve - Current pO2]: '+str(heav))
basic = UL_robustness(real, theta, real_shape)
print('UL Robustness [Cut of the end of the curve - Current pO2]: '+str(basic))
weight = LU_robustness(real, theta, real_shape)
print('LU Robustness [Cut of the end of the curve - Current pO2]: '+str(weight))
ll = LL_robustness(real, theta, real_shape)
print('LL Robustness [Cut of the end of the curve - Current pO2]: '+str(ll))


# Estimating the robustness w.r.t. pO2 at 37°
ideal = np.ones((mape.shape[0])) * mape[0,0]
real = mape[:,0]
real_shape = real.shape[0]
ideal_shape = ideal.shape[0]

heav = UU_robustness(real, theta, real_shape)
print('UU Robustness [Cut of the end of the curve - pO2 at 37°]: '+str(heav))
basic = UL_robustness(real, theta, real_shape)
print('UL Robustness [Cut of the end of the curve - pO2 at 37°]: '+str(basic))
weight = LU_robustness(real, theta, real_shape)
print('LU Robustness [Cut of the end of the curve - pO2 at 37°]: '+str(weight))
ll = LL_robustness(real, theta, real_shape)
print('LL Robustness [Cut of the end of the curve - pO2 at 37°]: '+str(ll))

UU Robustness [Cut of the end of the curve - Current pO2]: 0.14285714285714288
UL Robustness [Cut of the end of the curve - Current pO2]: 0.07260609127226521
LU Robustness [Cut of the end of the curve - Current pO2]: 0.2653061224415068
LL Robustness [Cut of the end of the curve - Current pO2]: 0.13650776782339885
UU Robustness [Cut of the end of the curve - pO2 at 37°]: 0.1904761904761903
UL Robustness [Cut of the end of the curve - pO2 at 37°]: 0.09020376226798277
LU Robustness [Cut of the end of the curve - pO2 at 37°]: 0.34467120183055056
LL Robustness [Cut of the end of the curve - pO2 at 37°]: 0.16696992813174405


<ipython-input-19-f7376d6114f0>:12: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  r = quad(integrand_b, 0, x0, args=(x0,err))


In [26]:
# Plot the accuracy function between the alteration range for the Current pO2
if DEBUG is True:
    plt.figure(figsize=(12, 7))
    c = plt.scatter(np.array(range(640,619,-1)),mape[:,1], marker='x', color='blue', )
    plt.xlabel('maximum value cancelled')
    plt.xticks(np.array(range(640,619,-1)))
    plt.yticks(np.array(range(-5,50,5)))
    plt.ylabel('MAPE error - current po2')
    plt.axhline(10, c='red', label = 'upper bound')
    plt.axhline(mape[0,1], c='green', label = 'nominal MAPE')


    ax = c.axes
    ax.invert_xaxis()
    plt.legend()
    plt.show()

In [27]:
# Plot the accuracy function between the alteration range for the pO2 at 37°
if DEBUG is True:
    plt.figure(figsize=(12, 7))
    c = plt.scatter(np.array(range(640,619,-1)),mape[:,0], marker='x', color='blue', )
    plt.xlabel('maximum value cancelled')
    plt.xticks(np.array(range(640,619,-1)))
    plt.yticks(np.array(range(-5,50,5)))
    plt.ylabel('MAPE error - pO2 at 37°')
    plt.axhline(10, c='red', label = 'upper bound')
    plt.axhline(mape[0,0], c='green', label = 'nominal MAPE')


    ax = c.axes
    ax.invert_xaxis()
    plt.legend()
    plt.show()

### 2. Clock Offset

In [28]:
# Configuration params
l_A = 0
u_A = 30
theta = 10

# Function to be used for introducing a clock offset
def clock_offset(onde_raw, offset):
    onde_new = np.copy(onde_raw)
    end = onde_new.shape[1]
    for i in range(end-offset):
        onde_new[:,end-1-i]=onde_new[:,end-1-i-offset]
    onde_new[:,:offset]=0
    return onde_new

In [29]:
# Alteration on all the curves and data analysis

# Prepare the resulting vectors as empty ones
mape = np.empty(((u_A-l_A) + 1, 2), dtype=np.float32)

# Apply all the levels of alteration
for j in range((u_A-l_A) + 1):
    onde_new = np.copy(onde_raw)

    # Alter all the curves
    onde_new = clock_offset(onde_new, j)  

    # Compute the parameters (5 average values) from each curve
    params = np.empty((len(onde_new), 5), dtype=np.float32)
    for i in range(len(onde_new)):
        try:
            params[i] = np.array(compute_params(onde_new[i]))
            assert np.any(np.isnan(params[i])) == False
        except:
            print("Parameter " + str(i) + " cannot be computed")
            continue

    # Test set + normalization
    new_data  = np.concatenate([df.TBlood.values.reshape(-1,1),params], axis = 1)
    test  = (new_data  - xmin) / (xmax - xmin)    

    # Prediction using the models
    if MODE == 0:
        pred = model.predict(test)
        y_pred_denorm  = denormalize(pred)
    else:
        # Using the parallel network as well
        y_pred_t0 = model.predict(test)
        y_pred_t1 = model1.predict(test)

        y_pred_t0_denorm  = denormalize(y_pred_t0)
        y_pred_t1_denorm  = denormalize(y_pred_t1)
        y_pred_denorm = (y_pred_t0_denorm + y_pred_t1_denorm)/2

    # Save the true values for evaluation
    pO2_true = np.concatenate([df.pO2_ABL1_37.values.reshape(-1,1), df.pO2_ABL1.values.reshape(-1,1)], axis=1)
   
    # Compute the MAPE
    mape[j,0]=mean_absolute_percentage_error(pO2_true[:, 0], y_pred_denorm[:, 0])
    mape[j,1]=mean_absolute_percentage_error(pO2_true[:, 1], y_pred_denorm[:, 1])

In [30]:
# Estimating the robustness w.r.t. Current pO2
ideal = np.ones((mape.shape[0])) * mape[0,1]
real = mape[:,1]
real_shape = real.shape[0]
ideal_shape = ideal.shape[0]

heav = UU_robustness(real, theta, real_shape)
print('UU Robustness [Clock offset - Current pO2]: '+str(heav))
basic = UL_robustness(real, theta, real_shape)
print('UL Robustness [Clock offset - Current pO2]: '+str(basic))
weight = LU_robustness(real, theta, real_shape)
print('LU Robustness [Clock offset - Current pO2]: '+str(weight))
ll = LL_robustness(real, theta, real_shape)
print('LL Robustness [Clock offset - Current pO2]: '+str(ll))

# Estimating the robustness w.r.t. pO2 at 37°
ideal = np.ones((mape.shape[0])) * mape[0,0]
real = mape[:,0]
real_shape = real.shape[0]
ideal_shape = ideal.shape[0]

heav = UU_robustness(real, theta, real_shape)
print('UU Robustness [Clock offset - pO2 at 37°]: '+str(heav))
basic = UL_robustness(real, theta, real_shape)
print('UL Robustness [Clock offset - pO2 at 37°]: '+str(basic))
weight = LU_robustness(real, theta, real_shape)
print('LU Robustness [Clock offset - pO2 at 37°]: '+str(weight))
ll = LL_robustness(real, theta, real_shape)
print('LL Robustness [Clock offset - pO2 at 37°]: '+str(ll))

UU Robustness [Clock offset - Current pO2]: 0.8709677419354835
UL Robustness [Clock offset - Current pO2]: 0.40099386716899754
LU Robustness [Clock offset - Current pO2]: 0.9833506764631899
LL Robustness [Clock offset - Current pO2]: 0.5231721099564837
UU Robustness [Clock offset - pO2 at 37°]: 0.8709677419354835
UL Robustness [Clock offset - pO2 at 37°]: 0.38599579974231485
LU Robustness [Clock offset - pO2 at 37°]: 0.9833506764631899
LL Robustness [Clock offset - pO2 at 37°]: 0.5171704632403897


<ipython-input-19-f7376d6114f0>:12: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  r = quad(integrand_b, 0, x0, args=(x0,err))


In [31]:
# Plot the accuracy function between the alteration range for the Current pO2
if DEBUG is True:
    plt.figure(figsize=(12, 7))
    c = plt.scatter(np.array(range(0,31,1)),mape[:,1], marker='x', color='blue', )
    plt.xlabel('Offset')
    plt.xticks(np.array(range(0,31,2)))
    plt.yticks(np.array(range(0,30,2)))
    plt.ylabel('MAPE error - current po2')
    plt.axhline(10, c='red', label = 'upper bound')
    plt.axhline(mape[0,1], c='green', label = 'MAPE_0')

    SMALL_SIZE = 8
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 25

    plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    ax = c.axes
    plt.legend()
    plt.show()

In [32]:
# Plot the accuracy function between the alteration range for the Current pO2
if DEBUG is True:
    plt.figure(figsize=(12, 7))
    c = plt.scatter(np.array(range(0,31,1)),mape[:,0], marker='x', color='blue', )
    plt.xlabel('Offset')
    plt.xticks(np.array(range(0,31,2)))
    plt.yticks(np.array(range(0,30,2)))
    plt.ylabel('MAPE error - pO2 at 37°')
    plt.axhline(10, c='red', label = 'upper bound')
    plt.axhline(mape[0,0], c='green', label = 'MAPE_0')

    SMALL_SIZE = 8
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 25

    plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


    ax = c.axes
    plt.legend()
    plt.show()

### 3. Cut of the peak 

In [33]:
# Configuration params
l_A = 1300
u_A = 4000
theta = 10
step = -10

# Function to be used for introducing a cut of the peak alteration
def peak_cut(onde_raw, threshold):
    onde_new = np.copy(onde_raw)
    for i in range(onde_new.shape[1]):
        index = np.where(onde_new[:,i]>threshold)
        onde_new[index,i] = threshold
    return onde_new

In [34]:
# Alteration on all the curves and data analysis

# Prepare the resulting vectors as empty ones
mape = np.empty((int((u_A-l_A)/np.abs(step)), 2), dtype=np.float32)

# Apply all the levels of alteration
values = range(u_A,l_A,step)

for j,s in enumerate(values):    
    # Alter all the curves
    onde_new = peak_cut(onde_raw, s)

    # Compute the parameters (5 average values) from each curve
    params = np.empty((len(onde_new), 5), dtype=np.float32)
    for i in range(len(onde_new)):
        try:
            params[i] = np.array(compute_params(onde_new[i]))
            assert np.any(np.isnan(params[i])) == False
        except:
            print("Parameter " + str(i) + " cannot be computed")
            continue

    # Test set + normalization
    new_data  = np.concatenate([df.TBlood.values.reshape(-1,1),params], axis = 1)
    test  = (new_data  - xmin) / (xmax - xmin)    

    # Prediction using the models
    if MODE == 0:
        pred = model.predict(test)
        y_pred_denorm  = denormalize(pred)
    else:
        # Using the parallel network as well
        y_pred_t0 = model.predict(test)
        y_pred_t1 = model1.predict(test)

        y_pred_t0_denorm  = denormalize(y_pred_t0)
        y_pred_t1_denorm  = denormalize(y_pred_t1)
        y_pred_denorm = (y_pred_t0_denorm + y_pred_t1_denorm)/2

    # Save the true values for evaluation
    pO2_true = np.concatenate([df.pO2_ABL1_37.values.reshape(-1,1), df.pO2_ABL1.values.reshape(-1,1)], axis=1)
   
    # Compute the MAPE
    mape[j,0]=mean_absolute_percentage_error(pO2_true[:, 0], y_pred_denorm[:, 0])
    mape[j,1]=mean_absolute_percentage_error(pO2_true[:, 1], y_pred_denorm[:, 1])

In [35]:
# Estimating the robustness w.r.t. Current pO2
ideal = np.ones((mape.shape[0])) * mape[0,1]
real = mape[:,1]
real_shape = real.shape[0]
ideal_shape = ideal.shape[0]

heav = UU_robustness(real, theta, real_shape)
print('UU Robustness [Cut of the peak - Current pO2]: '+str(heav))
basic = UL_robustness(real, theta, real_shape)
print('UL Robustness [Cut of the peak - Current pO2]: '+str(basic))
weight = LU_robustness(real, theta, real_shape)
print('LU Robustness [Cut of the peak - Current pO2]: '+str(weight))
ll = LL_robustness(real, theta, real_shape)
print('LL Robustness [Cut of the peak - Current pO2]: '+str(ll))

# Estimating the robustness w.r.t. pO2 at 37°
ideal = np.ones((mape.shape[0])) * mape[0,0]
real = mape[:,0]
real_shape = real.shape[0]
ideal_shape = ideal.shape[0]

heav = UU_robustness(real, theta, real_shape)
print('UU Robustness [Cut of the peak - pO2 at 37°]: '+str(heav))
basic = UL_robustness(real, theta, real_shape)
print('UL Robustness [Cut of the peak - pO2 at 37°]: '+str(basic))
weight = LU_robustness(real, theta, real_shape)
print('LU Robustness [Cut of the peak - pO2 at 37°]: '+str(weight))
ll = LL_robustness(real, theta, real_shape)
print('LL Robustness [Cut of the peak - pO2 at 37°]: '+str(ll))

UU Robustness [Cut of the peak - Current pO2]: 0.8370370416906961
UL Robustness [Cut of the peak - Current pO2]: 0.5246962500154823
LU Robustness [Cut of the peak - Current pO2]: 0.9734430740850389
LL Robustness [Cut of the peak - Current pO2]: 0.6348662952063278
UU Robustness [Cut of the peak - pO2 at 37°]: 0.8370370416906961
UL Robustness [Cut of the peak - pO2 at 37°]: 0.5330475809820328
LU Robustness [Cut of the peak - pO2 at 37°]: 0.9734430740850389
LL Robustness [Cut of the peak - pO2 at 37°]: 0.6459894651738256


<ipython-input-19-f7376d6114f0>:12: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  r = quad(integrand_b, 0, x0, args=(x0,err))


In [36]:
# Plot the accuracy function between the alteration range for the Current pO2
if DEBUG is True:
    plt.figure(figsize=(12, 7))
    c = plt.scatter(np.array(range(4000,1300,-10)),mape[:,1], marker='x', color='blue', )
    plt.xlabel('Cut of the peak')
    plt.xticks(np.array(range(1300,4000,200)))
    plt.yticks(np.array(range(3,30,2)))
    plt.ylabel('MAPE error - current po2')
    plt.axhline(10, c='red', label = 'upper bound')
    plt.axhline(mape[0,1], c='green', label = 'nominal MAPE')


    ax = c.axes
    ax.invert_xaxis()
    plt.legend()
    plt.show()

In [37]:
# Plot the accuracy function between the alteration range for the Current pO2
if DEBUG is True:
    plt.figure(figsize=(12, 7))
    c = plt.scatter(np.array(range(4000,1300,-10)),mape[:,0], marker='x', color='blue', )
    plt.xlabel('Cut of the peak')
    plt.xticks(np.array(range(1300,4000,200)))
    plt.yticks(np.array(range(3,30,2)))
    plt.ylabel('MAPE error - pO2 at 37°')
    plt.axhline(10, c='red', label = 'upper bound')
    plt.axhline(mape[0,0], c='green', label = 'nominal MAPE')


    ax = c.axes
    ax.invert_xaxis()
    plt.legend()
    plt.show()

### 4. Gaussian Noise

In [38]:
# Configuration params
l_A = 0
u_A = 50
theta = 10

# Function to be used for introducing a gaussian noise
def add_noise(onde_raw, standard_deviation):
    onde_new = np.copy(onde_raw)
    mean = 0
    for i in range(len(onde_new)):
        noise  = np.random.normal(mean, standard_deviation, size=onde_raw.shape[1])
        onde_new[i,:]= onde_new[i,:] + noise
    return onde_new  

In [39]:
# Alteration on all the curves and data analysis

# Prepare the resulting vectors as empty ones
mape = np.empty(((u_A-l_A) + 1, 2), dtype=np.float32)

# Apply all the levels of alteration
for j in range((u_A-l_A) + 1):
    onde_new = np.copy(onde_raw)

    # Alter all the curves
    onde_new = add_noise(onde_new, j)  

    # Compute the parameters (5 average values) from each curve
    params = np.empty((len(onde_new), 5), dtype=np.float32)
    for i in range(len(onde_new)):
        try:
            params[i] = np.array(compute_params(onde_new[i]))
            assert np.any(np.isnan(params[i])) == False
        except:
            print("Parameter " + str(i) + " cannot be computed")
            continue

    # Test set + normalization
    new_data  = np.concatenate([df.TBlood.values.reshape(-1,1),params], axis = 1)
    test  = (new_data  - xmin) / (xmax - xmin)    

    # Prediction using the models
    if MODE == 0:
        pred = model.predict(test)
        y_pred_denorm  = denormalize(pred)
    else:
        # Using the parallel network as well
        y_pred_t0 = model.predict(test)
        y_pred_t1 = model1.predict(test)

        y_pred_t0_denorm  = denormalize(y_pred_t0)
        y_pred_t1_denorm  = denormalize(y_pred_t1)
        y_pred_denorm = (y_pred_t0_denorm + y_pred_t1_denorm)/2

    # Save the true values for evaluation
    pO2_true = np.concatenate([df.pO2_ABL1_37.values.reshape(-1,1), df.pO2_ABL1.values.reshape(-1,1)], axis=1)
   
    # Compute the MAPE
    mape[j,0]=mean_absolute_percentage_error(pO2_true[:, 0], y_pred_denorm[:, 0])
    mape[j,1]=mean_absolute_percentage_error(pO2_true[:, 1], y_pred_denorm[:, 1])

In [40]:
# Estimating the robustness w.r.t. Current pO2
ideal = np.ones((mape.shape[0])) * mape[0,1]
real = mape[:,1]
real_shape = real.shape[0]
ideal_shape = ideal.shape[0]

heav = UU_robustness(real, theta, real_shape)
print('UU Robustness [Gaussian Noise - Current pO2]: '+str(heav))
basic = UL_robustness(real, theta, real_shape)
print('UL Robustness [Gaussian Noise - Current pO2]: '+str(basic))
weight = LU_robustness(real, theta, real_shape)
print('LU Robustness [Gaussian Noise - Current pO2]: '+str(weight))
ll = LL_robustness(real, theta, real_shape)
print('LL Robustness [Gaussian Noise - Current pO2]: '+str(ll))

# Estimating the robustness w.r.t. pO2 at 37°
ideal = np.ones((mape.shape[0])) * mape[0,0]
real = mape[:,0]
real_shape = real.shape[0]
ideal_shape = ideal.shape[0]

heav = UU_robustness(real, theta, real_shape)
print('UU Robustness [Gaussian Noise - pO2 at 37°]: '+str(heav))
basic = UL_robustness(real, theta, real_shape)
print('UL Robustness [Gaussian Noise - pO2 at 37°]: '+str(basic))
weight = LU_robustness(real, theta, real_shape)
print('LU Robustness [Gaussian Noise - pO2 at 37°]: '+str(weight))
ll = LL_robustness(real, theta, real_shape)
print('LL Robustness [Gaussian Noise - pO2 at 37°]: '+str(ll))

UU Robustness [Gaussian Noise - Current pO2]: 0.8823529411764708
UL Robustness [Gaussian Noise - Current pO2]: 0.35418619514737565
LU Robustness [Gaussian Noise - Current pO2]: 0.986159170159957
LL Robustness [Gaussian Noise - Current pO2]: 0.4874877207404803
UU Robustness [Gaussian Noise - pO2 at 37°]: 0.8431372549012515
UL Robustness [Gaussian Noise - pO2 at 37°]: 0.3451600058231515
LU Robustness [Gaussian Noise - pO2 at 37°]: 0.9753940788235412
LL Robustness [Gaussian Noise - pO2 at 37°]: 0.48394339719606144


<ipython-input-19-f7376d6114f0>:12: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  r = quad(integrand_b, 0, x0, args=(x0,err))


In [41]:
# Plot the accuracy function between the alteration range for the Current pO2
if DEBUG is True:
    plt.figure(figsize=(12, 7))
    c = plt.scatter(np.array(range(0,51,1)),mape[:,1], marker='x', color='blue', )
    plt.xlabel('Gaussian Noise')
    plt.xticks(np.array(range(0,51,10)))
    plt.yticks(np.array(range(3,30,2)))
    plt.ylabel('MAPE error - current po2')
    plt.axhline(10, c='red', label = 'upper bound')
    plt.axhline(mape[0,1], c='green', label = 'nominal MAPE')


    ax = c.axes
    plt.legend()
    plt.show()

In [42]:
# Plot the accuracy function between the alteration range for the pO2 at 37°
if DEBUG is True:
    plt.figure(figsize=(12, 7))
    c = plt.scatter(np.array(range(0,51,1)),mape[:,0], marker='x', color='blue', )
    plt.xlabel('Gaussian Noise')
    plt.xticks(np.array(range(0,51,10)))
    plt.yticks(np.array(range(3,30,2)))
    plt.ylabel('MAPE error - po2 at 37°C')
    plt.axhline(10, c='red', label = 'upper bound')
    plt.axhline(mape[0,0], c='green', label = 'nominal MAPE')


    ax = c.axes
    plt.legend()
    plt.show()

### 5. Amplification

In [43]:
# Configuration params
l_A = 1
u_A = 2
theta = 10
step = 0.1

# Function to be used for introducing an amplification alteration
def amplification(onde_raw, amplification):
    onde_new = np.copy(onde_raw)
    onde_new = onde_new * (amplification)
    return onde_new  

In [44]:
# Alteration on all the curves and data analysis

# Prepare the resulting vectors as empty ones
mape = np.empty((int((u_A-l_A)/step) + 1, 2), dtype=np.float32)

# Apply all the levels of alteration
values = np.arange(l_A, u_A+step, step)

for j,s in enumerate(values):
    # Alter all the curves
    onde_new = amplification(onde_raw, s)

    # Compute the parameters (5 average values) from each curve
    params = np.empty((len(onde_new), 5), dtype=np.float32)
    for i in range(len(onde_new)):
        try:
            params[i] = np.array(compute_params(onde_new[i]))
            assert np.any(np.isnan(params[i])) == False
        except:
            print("Parameter " + str(i) + " cannot be computed")
            continue

    # Test set + normalization
    new_data  = np.concatenate([df.TBlood.values.reshape(-1,1),params], axis = 1)
    test  = (new_data  - xmin) / (xmax - xmin)    

    # Prediction using the models
    if MODE == 0:
        pred = model.predict(test)
        y_pred_denorm  = denormalize(pred)
    else:
        # Using the parallel network as well
        y_pred_t0 = model.predict(test)
        y_pred_t1 = model1.predict(test)

        y_pred_t0_denorm  = denormalize(y_pred_t0)
        y_pred_t1_denorm  = denormalize(y_pred_t1)
        y_pred_denorm = (y_pred_t0_denorm + y_pred_t1_denorm)/2

    # Save the true values for evaluation
    pO2_true = np.concatenate([df.pO2_ABL1_37.values.reshape(-1,1), df.pO2_ABL1.values.reshape(-1,1)], axis=1)
   
    # Compute the MAPE
    mape[j,0]=mean_absolute_percentage_error(pO2_true[:, 0], y_pred_denorm[:, 0])
    mape[j,1]=mean_absolute_percentage_error(pO2_true[:, 1], y_pred_denorm[:, 1])

In [45]:
# Estimating the robustness w.r.t. Current pO2
ideal = np.ones((mape.shape[0])) * mape[0,1]
real = mape[:,1]
real_shape = real.shape[0]
ideal_shape = ideal.shape[0]

heav = UU_robustness(real, theta, real_shape)
print('UU Robustness [Amplification - Current pO2]: '+str(heav))
basic = UL_robustness(real, theta, real_shape)
print('UL Robustness [Amplification - Current pO2]: '+str(basic))
weight = LU_robustness(real, theta, real_shape)
print('LU Robustness [Amplification - Current pO2]: '+str(weight))
ll = LL_robustness(real, theta, real_shape)
print('LL Robustness [Amplification - Current pO2]: '+str(ll))

# Estimating the robustness w.r.t. pO2 at 37°
ideal = np.ones((mape.shape[0])) * mape[0,0]
real = mape[:,0]
real_shape = real.shape[0]
ideal_shape = ideal.shape[0]

heav = UU_robustness(real, theta, real_shape)
print('UU Robustness [Amplification - pO2 at 37°]: '+str(heav))
basic = UL_robustness(real, theta, real_shape)
print('UL Robustness [Amplification - pO2 at 37°]: '+str(basic))
weight = LU_robustness(real, theta, real_shape)
print('LU Robustness [Amplification - pO2 at 37°]: '+str(weight))
ll = LL_robustness(real, theta, real_shape)
print('LL Robustness [Amplification - pO2 at 37°]: '+str(ll))

UU Robustness [Amplification - Current pO2]: 1.0
UL Robustness [Amplification - Current pO2]: 0.466833533003435
LU Robustness [Amplification - Current pO2]: 1.0
LL Robustness [Amplification - Current pO2]: 0.5299627332647221
UU Robustness [Amplification - pO2 at 37°]: 1.0
UL Robustness [Amplification - pO2 at 37°]: 0.452648239514975
LU Robustness [Amplification - pO2 at 37°]: 1.0
LL Robustness [Amplification - pO2 at 37°]: 0.5340039993213734


<ipython-input-19-f7376d6114f0>:12: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  r = quad(integrand_b, 0, x0, args=(x0,err))


In [46]:
# Plot the accuracy function between the alteration range for the Current pO2
if DEBUG is True:
    plt.figure(figsize=(12, 7))
    c = plt.scatter(np.array(np.arange(l_A,u_A+step,step)),mape[:,1], marker='x', color='blue', )
    plt.xlabel('Amplification')
    plt.xticks(np.array(np.arange(l_A,u_A+step,step)))
    plt.yticks(np.array(np.arange(3,30,2)))
    plt.ylabel('MAPE error - current po2')
    plt.axhline(10, c='red', label = 'upper bound')
    plt.axhline(mape[0,1], c='green', label = 'nominal MAPE')


    ax = c.axes
    plt.legend()
    plt.show()

In [47]:
# Plot the accuracy function between the alteration range for the pO2 at 37°
if DEBUG is True:
    plt.figure(figsize=(12, 7))
    c = plt.scatter(np.array(np.arange(l_A,u_A+step,step)),mape[:,0], marker='x', color='blue', )
    plt.xlabel('Amplification')
    plt.xticks(np.array(np.arange(l_A,u_A+step,step)))
    plt.yticks(np.array(range(3,30,2)))
    plt.ylabel('MAPE error - po2 at 37°C')
    plt.axhline(10, c='red', label = 'upper bound')
    plt.axhline(mape[0,0], c='green', label = 'nominal MAPE')


    ax = c.axes
    plt.legend()
    plt.show()

### 6. Attenuation

In [48]:
# Configuration params
l_A = 0
u_A = 0.5
theta = 10
step = 0.1

# Function to be used for introducing an attenuation alteration
def attenuation(onde_raw, attenuation):
    onde_new = np.copy(onde_raw)
    onde_new *= (1 - attenuation)
    return onde_new

In [49]:
# Alteration on all the curves and data analysis

# Prepare the resulting vectors as empty ones
mape = np.empty((int((u_A-l_A)/step) + 1, 2), dtype=np.float32)

# Apply all the levels of alteration
values = np.arange(l_A, u_A+step, step)

for j,s in enumerate(values):
    # Alter all the curves
    onde_new = attenuation(onde_raw, s)

    # Compute the parameters (5 average values) from each curve
    params = np.empty((len(onde_new), 5), dtype=np.float32)
    for i in range(len(onde_new)):
        try:
            params[i] = np.array(compute_params(onde_new[i]))
            assert np.any(np.isnan(params[i])) == False
        except:
            print("Parameter " + str(i) + " cannot be computed")
            continue

    # Test set + normalization
    new_data  = np.concatenate([df.TBlood.values.reshape(-1,1),params], axis = 1)
    test  = (new_data  - xmin) / (xmax - xmin)    

    # Prediction using the models
    if MODE == 0:
        pred = model.predict(test)
        y_pred_denorm  = denormalize(pred)
    else:
        # Using the parallel network as well
        y_pred_t0 = model.predict(test)
        y_pred_t1 = model1.predict(test)

        y_pred_t0_denorm  = denormalize(y_pred_t0)
        y_pred_t1_denorm  = denormalize(y_pred_t1)
        y_pred_denorm = (y_pred_t0_denorm + y_pred_t1_denorm)/2

    # Save the true values for evaluation
    pO2_true = np.concatenate([df.pO2_ABL1_37.values.reshape(-1,1), df.pO2_ABL1.values.reshape(-1,1)], axis=1)
   
    # Compute the MAPE
    mape[j,0]=mean_absolute_percentage_error(pO2_true[:, 0], y_pred_denorm[:, 0])
    mape[j,1]=mean_absolute_percentage_error(pO2_true[:, 1], y_pred_denorm[:, 1])

In [50]:
# Estimating the robustness w.r.t. Current pO2
ideal = np.ones((mape.shape[0])) * mape[0,1]
real = mape[:,1]
real_shape = real.shape[0]
ideal_shape = ideal.shape[0]

heav = UU_robustness(real, theta, real_shape)
print('UU Robustness [Attenuation - Current pO2]: '+str(heav))
basic = UL_robustness(real, theta, real_shape)
print('UL Robustness [Attenuation - Current pO2]: '+str(basic))
weight = LU_robustness(real, theta, real_shape)
print('LU Robustness [Attenuation - Current pO2]: '+str(weight))
ll = LL_robustness(real, theta, real_shape)
print('LL Robustness [Attenuation - Current pO2]: '+str(ll))

# Estimating the robustness w.r.t. pO2 at 37°
ideal = np.ones((mape.shape[0])) * mape[0,0]
real = mape[:,0]
real_shape = real.shape[0]
ideal_shape = ideal.shape[0]

heav = UU_robustness(real, theta, real_shape)
print('UU Robustness [Attenuation - pO2 at 37°]: '+str(heav))
basic = UL_robustness(real, theta, real_shape)
print('UL Robustness [Attenuation - pO2 at 37°]: '+str(basic))
weight = LU_robustness(real, theta, real_shape)
print('LU Robustness [Attenuation - pO2 at 37°]: '+str(weight))
ll = LL_robustness(real, theta, real_shape)
print('LL Robustness [Attenuation - pO2 at 37°]: '+str(ll))

UU Robustness [Attenuation - Current pO2]: 1.0
UL Robustness [Attenuation - Current pO2]: 0.43560619354248037
LU Robustness [Attenuation - Current pO2]: 1.0
LL Robustness [Attenuation - Current pO2]: 0.5581675291061401
UU Robustness [Attenuation - pO2 at 37°]: 0.8333333333333333
UL Robustness [Attenuation - pO2 at 37°]: 0.40087769031524656
LU Robustness [Attenuation - pO2 at 37°]: 0.9722222222222224
LL Robustness [Attenuation - pO2 at 37°]: 0.5459218462308248


In [51]:
# Plot the accuracy function between the alteration range for the Current pO2
if DEBUG is True:
    plt.figure(figsize=(12, 7))
    c = plt.scatter(np.array(np.arange(l_A,u_A+step,step)),mape[:,1], marker='x', color='blue', )
    plt.xlabel('Attenuation')
    plt.xticks(np.array(np.arange(l_A,u_A+step,step)))
    plt.yticks(np.array(np.arange(3,30,2)))
    plt.ylabel('MAPE error - current po2')
    plt.axhline(10, c='red', label = 'upper bound')
    plt.axhline(mape[0,1], c='green', label = 'nominal MAPE')


    ax = c.axes
    plt.legend()
    plt.show()

In [52]:
# Plot the accuracy function between the alteration range for the pO2 at 37°
if DEBUG is True:
    plt.figure(figsize=(12, 7))
    c = plt.scatter(np.array(np.arange(l_A,u_A+step,step)),mape[:,0], marker='x', color='blue', )
    plt.xlabel('Attenuation')
    plt.xticks(np.array(np.arange(l_A,u_A+step,step)))
    plt.yticks(np.array(range(3,30,2)))
    plt.ylabel('MAPE error - po2 at 37°C')
    plt.axhline(10, c='red', label = 'upper bound')
    plt.axhline(mape[0,0], c='green', label = 'nominal MAPE')


    ax = c.axes
    plt.legend()
    plt.show()